<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


После загрузки данных установлено отсутствие пропусков, что означает, что можно продолжать работу с данными.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** b. Не изменится.

**Обоснование:** $$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T X P)^{-1} (XP)^T y = (P^T (X^T X) P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y=P^{-1}w
$$
$$
a' = X\cdot P\cdot w' = X\cdot P\cdot P^{-1}w = X\cdot w
$$
$$
a' = a
$$

Эти выражения идентичны друг другу, и векторы, которые были использованы для предсказания, совпадают.

Параметры линейной регрессии в исходной задаче и в преобразованной связаны следующим образом:
$$
w_P = w P^{-1}
$$
где $w$ - вектор весов линейной регрессии в исходной задаче, $w_P$ - вектор весов линейной регрессии в преобразованной задаче, а $P$ - матрица, на которую умножаются признаки.

Это соотношение можно получить из формулы для вычисления вектора весов $w$:
$$
w = (X^T X)^{-1} X^T y
$$

Если вместо матрицы признаков $X$ использовать преобразованную матрицу признаков $XP$, то формула для вектора весов будет выглядеть следующим образом:

$$
w_P = ((XP)^T XP)^{-1} (XP)^T y
$$

Подставляя $XP = X P$, получаем:

$$
w_P = (P^T X^T XP)^{-1} P^T X^T y
$$

Используя свойство обратной матрицы ($A A^{-1} = A^{-1} A = I$), можно преобразовать выражение:

$$
w_P = P^{-1}( X^T X)^{-1} X^T y = wP^{-1}
$$

Таким образом, параметры линейной регрессии в исходной задаче и в преобразованной связаны матрицей преобразования признаков $P$ и умножением вектора весов линейной регрессии $w$ на обратную матрицу $P^{-1}$.

## Алгоритм преобразования

**Алгоритм**

Сначала будет выполнен следующий набор действий: изначально отделим целевой признак, затем произведем разбиение выборки и обучим модель на первоначальных данных. Затем оценим метрику R2.

Далее перейдем к следующему этапу, который будет состоять в создании случайной матрицы и ее инвертировании. После этого умножим матрицу с данными признаков на случайную матрицу. Затем отделим целевой признак, разобьем выборки и обучим модель, а затем оценим метрику R2.

Наконец, сравним показатель R2, полученный после выполнения первого набора действий, с показателем R2, полученным после выполнения второго набора действий.

**Обоснование**

При умножении матриц мы комбинируем несколько наборов данных и записываем результат в новую матрицу, однако вес каждого вектора останется таким же, как и в изначальном наборе.

## Проверка алгоритма

Реализация до

In [6]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
model = LinearRegression()
model = model.fit(features_train, target_train)
predictions = model.predict(features_test)

In [8]:
r2_score(target_test, predictions)

0.42375177725680335

Реализация после

In [9]:
matrix = np.random.rand(4,4)
matrix_inverse = np.linalg.inv(matrix)

In [10]:
matrix @ matrix_inverse

array([[ 1.00000000e+00, -2.12830153e-16,  7.95069816e-16,
         5.27647712e-16],
       [ 1.59213100e-17,  1.00000000e+00,  1.05951695e-15,
        -7.63272459e-16],
       [-6.60869460e-18, -1.28550517e-17,  1.00000000e+00,
         3.79463140e-16],
       [ 8.48466867e-17,  1.01586161e-17,  4.34445970e-16,
         1.00000000e+00]])

In [11]:
tests = features.values @ matrix

In [12]:
tests_train, tests_test, target_train, target_test = train_test_split(tests, target, test_size=0.4, random_state=12345)
model1 = LinearRegression()
model1 = model1.fit(tests_train, target_train)
predictions1 = model1.predict(tests_test)

In [13]:
r2_score(target_test, predictions1)

0.42375177725689706

Метрика R2 осталась практически неизменной

**Общий вывод**

Были рассмотрены данные, не содержащие пропусков, аномалий и других отклонений, которые могли бы помешать дальнейшей работе. Мы установили, что если признаки умножаются на обратимую матрицу, то метрика R2 не изменится. Был разработан алгоритм, который включал сравнение метрики R2 до и после преобразования. Практическое применение этого алгоритма доказало, что метрика R2 остается неизменной при умножении признаков на обратимую матрицу.